In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from __future__ import print_function
import math

data_file = 'final_fare_all.txt'
testData_percent = 0.2

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("RandomForestRegressor").getOrCreate()

data = MLUtils.loadLibSVMFile(sc, data_file)
(trainData, testData) = data.randomSplit([1-testData_percent, testData_percent])

##################
cat_fea_info_1 = {}
cat_fea_info_1[0] = 266
cat_fea_info_1[1] = 24
cat_fea_info_1[2] = 2

cat_fea_info_2 = {}
cat_fea_info_2[0] = 266
#cat_fea_info_2[1] = 24
cat_fea_info_2[2] = 2
###################

testRMSEs = []
parameters = []
iter = 0

for cat_fea_info in [cat_fea_info_1, cat_fea_info_2]:
    for num_trees in range(100, 501, 100):
        for feature_subset_mode in ['auto', 'all', 'log2', 'onethird']:
            for max_depth in range(2, 6):
                for max_bins in [266, 300, 400, 500]:
                
                    iter += 1
                    print(iter)
                    parameters.append([cat_fea_info, num_trees, feature_subset_mode, max_depth, max_bins])
                    
                    # Train RandomForest Regressor
                    RF = RandomForest.trainRegressor(data=trainData, categoricalFeaturesInfo=cat_fea_info, 
                                                        numTrees=num_trees, featureSubsetStrategy=feature_subset_mode,
                                                        impurity='variance', maxDepth=max_depth, maxBins=max_bins)
                                      
                    # Test model and compute Root Mean Square Error
                    predictions = RF.predict(testData.map(lambda row: row.features))
                    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
                    testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() / float(testData.count())
                    testRMSE = math.sqrt(testMSE)
                    print('Test Root Mean Squared Error on ' + data_file + ' = ' + str(testRMSE))
                    #print('Learned regression forest model:')
                    #print(model.toDebugString())

                    testRMSEs.append(testRMSE)

#Best model    
min_RMSE = min(testRMSEs)
print('Minimum Test RMSE is ' + str(min_RMSE))
#index = testRMSEs.index(min(testRMSEs))
best_parameters = parameters[testRMSEs.index(min(testRMSEs))]
print('Parameters of the best model is ' + str(best_parameters))

1
Test Root Mean Squared Error on final_fare_all.txt = 7.802136344125191
2
Test Root Mean Squared Error on final_fare_all.txt = 7.792350870829626
3
Test Root Mean Squared Error on final_fare_all.txt = 7.793420034563718
4
